# Análise Global de Diversidade

In [119]:
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")

In [120]:
ANOS = [2015, 2016, 2017, 2018, 2019]
UNIVERSIDADES = ['IFBA', 'UFBA', 'UEFS', 'UNEB', 'IFBAIANO', 'UESB', 'UNIVASF', 'UESC']

In [121]:
data_frame_diversidade = pd.read_csv('dataset/alunos_curso_2015_2019_tic_BA.csv')
data_frame_diversidade.head()

,NU_ANO_CENSO,CO_IES,TP_CATEGORIA_ADMINISTRATIVA,TP_ORGANIZACAO_ACADEMICA,CO_CURSO,CO_CURSO_POLO,TP_TURNO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,...,CO_PAIS_DESTINO,IN_MATRICULA,IN_CONCLUINTE,IN_INGRESSO_TOTAL,IN_INGRESSO_VAGA_NOVA,IN_INGRESSO_PROCESSO_SELETIVO,NU_ANO_INGRESSO,CO_UF,NO_CURSO,SG_IES
0,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA
1,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA
2,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA
3,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA
4,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA


In [122]:
replace_map_cor_raca = {
    'TP_COR_RACA':{
        0: 'Não quis declarar',  
        1: 'Branca',
        2: 'Preta',
        3: 'Parda',
        4: 'Amarela',
        5: 'Indígena',
        9: 'Sem informação'
    }
}

In [123]:
replace_map_sexo = {
    'TP_SEXO':{
        1: 'Feminino',
        2: 'Masculino'
    }
}

In [124]:
data_frame_diversidade.replace(replace_map_cor_raca, inplace=True)

In [125]:
data_frame_diversidade.replace(replace_map_sexo, inplace=True)

In [126]:
def calculo_desvio_padrao_colunas_categoricas(coluna, dataframe):
    return round(dataframe[coluna].std(), 4)

In [127]:
def min_max_scaler(dataframe, column_name):
    min_param = dataframe[column_name].min()
    max_param = dataframe[column_name].max()
    return (dataframe[column_name] - min_param) / (max_param- min_param)
        

In [128]:
def calcula_diversidade_global(dataframe_execucao, coluna_principal, coluna_quantidade):
    
    std_geral = {'IFBA': [], 'UFBA': [], 'UEFS': [], 'UNEB': [], 'IFBAIANO': [], 'UESB': [], 'UNIVASF': [], 'UESC': []}
    dataframe_temp = dataframe_execucao.groupby(['SG_IES', 'NU_ANO_CENSO'])
    dataframe_temp = dataframe_temp[coluna_principal].value_counts().to_frame(name = coluna_quantidade).reset_index()
    for ano in ANOS:
        current_dataframe = dataframe_temp[dataframe_temp['NU_ANO_CENSO'] == ano]
        current_dataframe[coluna_quantidade] = min_max_scaler(current_dataframe, coluna_quantidade)
        for universidade in UNIVERSIDADES:        
            current_dataframe_temp = current_dataframe[current_dataframe['SG_IES'] == universidade] 
            std_result = calculo_desvio_padrao_colunas_categoricas(coluna_quantidade, current_dataframe_temp)    
            std_geral[universidade].append(std_result)
    return std_geral

## Análise da dimensão Gênero

In [129]:
std_genero = calcula_diversidade_global(data_frame_diversidade, 'TP_SEXO', 'TP_SEXO_QUANTIDADE' )

## Análise da dimensão Cor e Raça


In [98]:
data_frame_diversidade['TP_COR_RACA'].value_counts()

Não quis declarar    8399
Parda                5518
Preta                2654
Branca               1615
Amarela               107
Indígena               66
Sem informação         55
Name: TP_COR_RACA, dtype: int64

In [133]:
dataframe_temp = data_frame_diversidade[(data_frame_diversidade['TP_COR_RACA'] != 'Não quis declarar') & (data_frame_diversidade['TP_COR_RACA'] != 'Sem informação')]
dataframe_temp['TP_COR_RACA'].value_counts()

Parda       5518
Preta       2654
Branca      1615
Amarela      107
Indígena      66
Name: TP_COR_RACA, dtype: int64

In [134]:
std_cor_raca = calcula_diversidade_global(data_frame_diversidade, 'TP_COR_RACA', 'TP_COR_RACA_QUANTIDADE' )